In [1]:
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tflearn
import tensorflow as tf
import random
import json

# nltk.download('punkt') (if running for first time on your machine)

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
# Prepare a stemmer
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [3]:
# Load the data
with open('intents.json') as file:
    data = json.load(file)

In [4]:
words = [] # possible unique words
labels = [] # possible unique labels (tags)
docs_x = [] # Pattern inputs
docs_y = [] # Corresponding tags for docs_x patterns

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        pattern_wrds = nltk.word_tokenize(pattern) # breaks up string into list of words
        words.extend(pattern_wrds) # extend() adds all elements of an iterable to the end of the list
        docs_x.append(pattern_wrds) # If a pattern > one word then stemmer will just lowercase it
        docs_y.append(intent["tag"])

    if intent["tag"] not in labels:
        labels.append(intent["tag"])

In [5]:
# Stem words and convert to lowercase
words = [stemmer.stem(w) for w in words]

In [6]:
# Use set to remove duplicates and then sort
words = sorted(list(set(words)))
labels = sorted(labels)

In [7]:
# Using one-hot encoding to prepare data for model
training = []
output = []
out_empty = [0 for _ in range(len(labels))]

for i, doc in enumerate(docs_x):
    bag = []

    # Stem each word in pattern and remove common punctuation symbols
    wrds = [stemmer.stem(w) for w in doc if w not in "?!.,'"]

    for w in words:
        if w in doc:
            bag.append(1) # put 1 if word is in pattern
        else:
            bag.append(0) # 0 if not

    output_row = out_empty[:]
    output_row[labels.index(docs_y[i])] = 1

    training.append(bag)
    output.append(output_row)

In [8]:
# Convert lists into numpy arrays so that they can be given as inputs to the model
x = np.array(training)
Y = np.array(output)

In [9]:
x[:3]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]])

In [10]:
Y[:3]

array([[0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 0]])

In [11]:
# Building Neural Net
# Input layer has len(x[0]) neurons (# of unique words)
net = tflearn.input_data(shape=[None, len(x[0])])
net = tflearn.fully_connected(net, 8) # hidden layer with 8 neurons
net = tflearn.fully_connected(net, 8) # hidden layer with 8 neurons
# Output layer has len(Y[0]) neurons (# of unique pattern tags/labels)
net = tflearn.fully_connected(net, len(Y[0]), activation="softmax")
net = tflearn.regression(net)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [12]:
model = tflearn.DNN(net)

model.fit(training, output, n_epoch=780, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 3119  | total loss: 0.20132 | time: 0.007s
| Adam | epoch: 780 | loss: 0.20132 - acc: 0.9316 -- iter: 24/26
Training Step: 3120  | total loss: 0.19856 | time: 0.009s
| Adam | epoch: 780 | loss: 0.19856 - acc: 0.9385 -- iter: 26/26
--
INFO:tensorflow:c:\Users\iliarom\Desktop\GitHub\Resume-Chat-Bot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
